In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../bbgfactor/rty_2014_smallfeatures.csv', parse_dates = ['DATE'])

In [3]:
dates = df.DATE.drop_duplicates().sort_values()

In [4]:
medates = dates.groupby(dates.dt.to_period('M')).tail(1)

In [5]:
df.index

RangeIndex(start=0, stop=3854316, step=1)

In [6]:
mdf = df[df.DATE.isin(medates)]

In [11]:
mdf.groupby(['DATE']).count()

,ID,forward_return,stock_excess_return,market_cap,avg_volm_to_cap,volume,volatility
DATE,,,,,,,
2013-12-31,932,932,0,932,0,932,0
2014-01-31,938,938,938,938,0,938,0
2014-02-28,941,941,941,941,0,941,0
2014-03-31,954,954,954,954,0,954,0
2014-04-30,960,960,960,960,960,960,960
...,...,...,...,...,...,...,...
2024-07-31,1901,1901,1901,1901,1901,1901,1901
2024-08-30,1902,1902,1902,1902,1902,1902,1902
2024-09-30,1909,1909,1909,1909,1909,1909,1909


In [7]:
# here rank is forward_return 
# group id  is date 
# you dont see the ID ( ticker column)

In [17]:
X_train_inds = mdf[mdf.DATE <= '2020-12-31'].index
X_test_inds = mdf[mdf.DATE > '2020-12-31'].index

In [18]:
# from sklearn.model_selection import GroupShuffleSplit

# gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(mdf, groups=mdf['DATE'])

# X_train_inds, X_test_inds = next(gss)

train_data= df.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['DATE','ID','forward_return'])]
y_train = train_data.loc[:, train_data.columns.isin(['forward_return'])]

groups = train_data.groupby('DATE').size().to_frame('size')['size'].to_numpy()

test_data= df.iloc[X_test_inds]

#We need to keep the id for later predictions
X_test = test_data.loc[:, ~test_data.columns.isin(['DATE','ID','forward_return'])]
y_test = test_data.loc[:, test_data.columns.isin(['forward_return'])]

In [ ]:
import xgboost as xgb

model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )

model.fit(X_train, y_train, group=groups, verbose=True)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=0.9,
          device=None, early_stopping_rounds=None, enable_categorical=False,
          eta=0.05, eval_metric=None, feature_types=None, gamma=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=0.1, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=110,
          n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
## the input for ncdg has to be an integer variable. 
# can just rank the assets 1 to n 
# or can rank them in buckets of 1%, so 1 to 100 
# leaving this error message here to fix this and try later 

import xgboost as xgb

model = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    random_state=42,

    objective='rank:ndcg',
    lambdarank_num_pair_per_sample=10,    # Compare each item to 8 others
    lambdarank_pair_method='topk',       # Focus on the top 8
    
    eta=0.05,
    learning_rate=0.1,
    
    colsample_bytree=0.9, 
    subsample=0.75,
     
    max_depth=6, 
    n_estimators=110, 
    
    )

model.fit(X_train, y_train, group=groups, verbose=True)

XGBoostError: [16:34:51] /Users/runner/work/xgboost/xgboost/src/common/ranking_utils.h:353: Check failed: label_is_integer: When using relevance degree as target, label must be either 0 or positive integer.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000002cbbf4428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000002cbd1e27c xgboost::ltr::NDCGCache::InitOnCPU(xgboost::Context const*, xgboost::MetaInfo const&) + 1296
  [bt] (2) 3   libxgboost.dylib                    0x00000002cbe2e1ec xgboost::ltr::NDCGCache::NDCGCache(xgboost::Context const*, xgboost::MetaInfo const&, xgboost::ltr::LambdaRankParam const&) + 172
  [bt] (3) 4   libxgboost.dylib                    0x00000002cbe4c104 xgboost::obj::LambdaRankObj<xgboost::obj::LambdaRankNDCG, xgboost::ltr::NDCGCache>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::linalg::Tensor<xgboost::detail::GradientPairInternal

In [ ]:
def predict(model, df):
    
    return model.predict(df.loc[:, ~df.columns.isin(['DATE','ID','forward_return'])])



In [25]:
def predict(model, df):

    idx = df['ID']

    y_pred = model.predict(df.loc[:, ~df.columns.isin(['DATE','ID','forward_return'])])
       
    return pd.Series(y_pred, index=idx)



In [26]:
test_data

,ID,DATE,forward_return,stock_excess_return,market_cap,avg_volm_to_cap,volume,volatility
463,2551003D US Equity,2021-01-29,0.104421,-0.195420,-0.030810,0.044557,0.140265,0.504745
482,2551003D US Equity,2021-02-26,-0.020833,-1.480970,0.023531,0.003813,0.636621,0.019728
505,2551003D US Equity,2021-03-31,-0.167282,-0.781010,0.119172,-0.024087,0.119169,0.528289
526,2551003D US Equity,2021-04-30,-0.150276,-0.273459,0.395619,0.161973,0.160161,0.727726
546,2551003D US Equity,2021-05-28,-0.108220,-0.199008,0.340448,0.334509,1.280993,0.968266
...,...,...,...,...,...,...,...,...
3854247,ZYXI UW Equity,2024-07-31,-0.008889,2.422115,-0.811078,-0.151472,-0.329443,0.095216
3854269,ZYXI UW Equity,2024-08-30,0.058524,-0.857126,-0.809152,-0.135855,-0.206361,0.058424
3854289,ZYXI UW Equity,2024-09-30,-0.026961,0.214528,-0.780932,-0.138979,-0.476264,-0.059987
3854312,ZYXI UW Equity,2024-10-31,-0.066049,-0.580682,-0.769612,-0.240809,-0.359622,-0.344922


In [27]:

  
predictions = (test_data.groupby('DATE')
               .apply(lambda x: predict(model, x)))

/var/folders/sw/kl2bvz9d5275zz0p278kz9nc0000gn/T/ipykernel_3587/915525311.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: predict(model, x)))


In [28]:
predictions

DATE        ID                
2021-01-29  2551003D US Equity   -0.629156
            AAMI UN Equity       -0.773603
            AAOI UQ Equity        0.110508
            AAT UN Equity        -0.650097
            ABCB UN Equity       -0.631906
                                    ...   
2024-11-05  ZURA UR Equity        0.314760
            ZVRA UW Equity       -0.135469
            ZWS UN Equity        -1.072112
            ZYME UW Equity       -0.737189
            ZYXI UW Equity       -0.510706
Length: 84809, dtype: float32